# 数据清洗

1. 合并指定日期内的 Sales 数据
2. 根据业务逻辑，选择合适的字段
3. 筛选出 'sub_cat_2_name_chi' == '隱形眼鏡' 的记录
4. 输出两份DataFrame：1份不剔除 'estate_name_chi' 为 NULL，但保证 'district' 不为 NULL，用来分析地区间的情况；另一份保证 'estate_name_chi' 不为 NULL，以便后续精确到小区
5. 由于 'order_value' 为一个价格区间，取其均值代表这一单的花费
6. 将 'district' 映射到标准的 18区 的描述，以便在tableau中画图

In [2]:
import pandas as pd
import numpy as np
import os 
import warnings
warnings.filterwarnings('ignore')

## 1 合并指定日期内的 Sales 数据

In [3]:
# root_path = 'data'
root_path = '/Users/yangyong/Documents/文稿/HKUST/1_Spring_2022/MSDM5005/proj 2/data'


f_paths = []
for root, dirs, files in os.walk(root_path):
    for name in files:
        if os.path.splitext(name)[1] == '.csv':
            f_paths.append(os.path.join(root, name))

f_paths.sort()
print(f'# of csv: {len(f_paths)}')

# of csv: 7


In [4]:
df_lst = []
for i in range(len(f_paths)):
    if i % round(len(f_paths)/10) == 0:
        print(f'processing: {round((i/round(len(f_paths)))*100)}%...')
    if i == len(f_paths)-1:
        print('done!')
    df = pd.read_csv(f_paths[i])
    df_lst.append(df)

print(f'# of csv: {len(df_lst)}')
data = pd.concat(df_lst)
print(f'the shape of data is {data.shape}')

processing: 0%...
processing: 14%...
processing: 29%...
processing: 43%...
processing: 57%...
processing: 71%...
processing: 86%...
done!
# of csv: 7
the shape of data is (1388633, 112)


In [5]:
data.sample(5)

,hktv_ad_id,membership_level,order_number,additional_parent_order,order_date,order_time_range,delivery_date,delivery_timeslot,app_type,device_type,...,storage_type,color,size,weight,height,length,width,manufacturer_country_en,manufacturer_country_chi,removal_service_requested
8498,sZSeoSJi/eXU/AoGry/A12fa2XA/J1Mtcpa5pxTNcwo=,NORMAL,f3u+PmCfojdD47nh1Lp2vfrMvfsdxW/24LVQ5Uz9l0A=,NaN,2022-04-24,18-21,2022-04-24,14:00 - 18:00,MAIN,NaN,...,M,NaN,NaN,1100g,450mm,550mm,400mm,China,中國,N
75989,1eTEhUXl5Be4z9opKJLj2EUJo/Hu8z7R4FpSUWj8nbk=,GOLDVIP,YTzmuv1eyyDZ56/OsLapK5ONTBDdZO/LXJMb54Hcxto=,NaN,2022-04-25,09-12,2022-04-27,18:00 - 21:00,MAIN,iOS,...,H,NaN,NaN,598g,80mm,200mm,230mm,China,中國,N
93779,+GU7rYEwc+YPIp5NdkfnACsep84/yV/db1KuDMd62Ks=,GOLDVIP,NTevIAWRK/hEVyWuYY3t15RqpIUUHEIzLtUZomGj8MM=,NaN,2022-04-20,00-09,2022-04-21,18:00 - 21:00,MAIN,iOS,...,H,NaN,NaN,240g,70mm,170mm,260mm,Vietnam,越南,N
117725,8mqEPxktlu8tgNMEGVjIU89Yib92qLnHS/uuNtK9+X4=,GOLDVIP,f4ixL8s27sjvUfiTg85cLa0SAsgo5PxUwkTwBhH2sHc=,NaN,2022-04-20,21-24,2022-04-23,14:00 - 18:00,MAIN,Android,...,H,NaN,NaN,1625g,111mm,95mm,199mm,Hong Kong,香港,N
197375,wPh4qYprVJQQVYUauU1jmPInnUFiV6cQdXuzM4pgAMU=,VIP,8NbCIAhmgHQNiyU+ACgnXxMCANWthHTdXeEO5pGleP0=,NaN,2022-04-22,21-24,2022-04-26,18:00 - 21:00,MAIN,Android,...,H,NaN,NaN,588g,180mm,112mm,65mm,Korea,韓國,N


In [6]:
data.columns[:50]

Index(['hktv_ad_id', 'membership_level', 'order_number',
       'additional_parent_order', 'order_date', 'order_time_range',
       'delivery_date', 'delivery_timeslot', 'app_type', 'device_type',
       'sales_application', 'payment_gateway', 'card_class', 'card_issuer',
       'card_type', 'area', 'district', 'delivery_district', 'delivery_zone',
       'estate_name_en', 'estate_name_chi', 'housing_type', 'order_value',
       'delivery_cost', 'merchant_delivery_fee', 'total_discounts',
       'credit_applied', 'refund_amount', 'refund_delivery_amount',
       'cash_voucher_code', 'cash_voucher_value', 'cash_voucher_applied_value',
       'paid_voucher_code', 'paid_voucher_value', 'paid_voucher_applied_value',
       'free_delivery_voucher', 'free_delivery_applied_value', 'campaign_code',
       'order_mall_level_discount_sum', 'order_mall_level_promotion_codes',
       'order_mall_level_promotion_amounts',
       'order_redemption_promotion_codes',
       'order_redemption_promotion

In [7]:
data.columns[50:112]

Index(['product_id', 'sku_id', 'primary_sku_id', 'bundle_set', 'sku_name_en',
       'sku_name_chi', 'brand_en', 'brand_chi', 'quantity', 'total_price',
       'primary_category', 'primary_category_name_en',
       'primary_category_name_chi', 'sub_cat_1_name_en', 'sub_cat_1_name_chi',
       'sub_cat_2_name_en', 'sub_cat_2_name_chi', 'sub_cat_3_name_en',
       'sub_cat_3_name_chi', 'sub_cat_4_name_en', 'sub_cat_4_name_chi',
       'primary_store', 'store_id', 'store_name_en', 'store_name_chi',
       'virtual_store_id', 'virtual_store_name_en', 'virtual_store_name_chi',
       'loyalty_point_rate', 'sku_level_promotion_type',
       'sku_level_promotion_code', 'sku_level_promotion_amount',
       'store_level_promotion_type', 'store_level_promotion_code',
       'store_level_promotion_amount', 'store_discount_rate',
       'mall_level_promotion_type', 'mall_level_promotion_code',
       'mall_level_promotion_amount', 'gift_promotion_id', 'rm_code',
       'delivery_mode', 'delivery_i

In [8]:
data['delivery_date'].unique()

array(['2022-04-21', '2022-04-20', '2022-04-23', '2022-04-24',
       '2022-04-22', '2022-05-02', '2022-04-25', '2022-04-27',
       '2022-04-19', '2022-04-29', '2022-04-28', '2022-04-30',
       '2022-04-26', '2022-05-04', '2022-05-03', '2022-05-01', nan,
       '2022-05-05', '2022-05-07', '2022-05-12', '2022-05-06',
       '2022-05-15', '2022-05-09', '2022-05-08', '2022-05-10'],
      dtype=object)

## 2. 选择字段，筛选 '隱形眼鏡' 的记录，处理空值

In [9]:
cols = ['membership_level'
      , 'order_date'
      , 'delivery_date'
      , 'app_type'
      , 'device_type'
      , 'area'
      , 'district'
      , 'estate_name_en'
      , 'estate_name_chi'
      , 'housing_type'
      , 'order_value'
      , 'delivery_cost'
      , 'total_discounts'
      , 'product_id'
      , 'sku_id'
      , 'sku_name_en'
      , 'sku_name_chi'
      , 'brand_en'
      , 'brand_chi'
      , 'primary_category_name_chi'
      , 'sub_cat_1_name_chi'
      , 'sub_cat_2_name_chi'
      , 'sub_cat_3_name_chi'
      , 'sub_cat_4_name_chi'
      , 'store_name_chi'
      , 'pickup_date'
      , 'size'
      , 'weight'
      , 'height'
      , 'width'
      , 'manufacturer_country_chi']

In [10]:
col = 'sub_cat_2_name_chi'
sub_cat_2 = '隱形眼鏡'

In [11]:
# # 'district' 不为 NULL
# df1 = data[ (data[col] ==  sub_cat_2) & (data['district'].notna())][cols]
# df1.shape

In [12]:
# delivery_date = df1['delivery_date'].to_numpy()
# order_date = df1['order_date'].to_numpy()

# waiting_time = []
# for i in range(df1.shape[0]):
#     waiting_time.append(int(delivery_date[i][-2:]) - int(order_date[i][-2:]))

In [13]:
# 'district' 和 'estate_name_chi' 都不为 NULL
df2 = data[ (data[col] ==  sub_cat_2) & (data['district'].notna()) & (data['estate_name_chi'].notna())][cols]
df2.shape

(2073, 31)

In [14]:
# delivery_date = df2['delivery_date'].to_numpy()
# order_date = df2['order_date'].to_numpy()

# waiting_time = []
# for i in range(df2.shape[0]):
#     waiting_time.append(int(delivery_date[i][-2:]) - int(order_date[i][-2:]))

In [15]:
# df2['waiting_time'] = waiting_time

## 3. 取 'order_value' 中间值

In [16]:
def take_mean(arr):
    size = len(arr)
    res = np.zeros(size)
    for i in range(size):
        res[i] = np.mean(list(map(int, arr[i].split(" - "))))
    return res

In [17]:
# df1['mean_order_value'] = take_mean(df1['order_value'].to_numpy())

In [18]:
df2['mean_order_value'] = take_mean(df2['order_value'].to_numpy())

## 4. 将 'district' 映射到标准的 18区 的描述

In [19]:
CNW = "Central and West District"
ED = "Eastern District"
SD = "Southern District"
WC = "Wan Chai District"
KC = "Kowloon City District"
KTO = "Kwun Tong District"
SSP = "Sham Shui Po District"
WTS = "Wong Tai Sin District"
YTM = "Yau Tsim Mong District"
ID = "Island District"
KTI = "Kwai Tsing District"
ND = "North District"
SK = "Sai Kung District"
ST = "Sha Tin District"
TP = "Tai Po District"
TW = "Tsuen Wan District"
TM = "Tuen Mun District"
YL = "Yuen Long District"

district_dict = {'MA ON SHAN NT':ST, 
              'TSUEN WAN NT':TW, 
              'SAU MAU PING KLN':KTO,
              'HANG HAU NT':SK, 
              'BRAEMAR HILL HK':ED, 
              'STANLEY HK':SD,
              'KWUN TONG IND AREA KLN':KTO, 
              'YAU MA TEI KLN':YTM, 
              'FANLING NT':ND,
              'KWAI CHUNG NT':KTI, 
              'TAI PO NT':TP, 
              'HENG FA CHUEN HK':ED, 
              'QUARRY BAY HK':ED,
              'TAI WO NT':TP, 
              'PO LAM NT':SK, 
              'HUNG HOM KLN':KC, 
              'SHAM SHUI PO KLN':SSP,
              'ADMIRALTY HK':CNW, 
              'KOWLOON STATION KLN':KC, 
              'SHATIN NT':ST, 
              'YAU TONG KLN':KTO,
              'SHEUNG SHUI NT':ND, 
              'TAI KOO HK':ED, 
              'TSING YI NT':KTI, 
              'WONG TAI SIN KLN':WTS,
              'MEI FOO':SSP, 
              'MID-LEVELS HK':CNW, 
              'CAUSEWAY BAY HK':WC, 
              'SAI WAN HK':ED,
              'WAN CHAI HK':WC, 
              'TO KWA WAN KLN':KC, 
              'TSZ WAN SHAN KLN':WTS,
              'TIN SHUI WAI NT':YL, 
              'SHAM TSENG NT':TW, 
              'KENNEDY TOWN HK':CNW,
              'TSIM SHA TSUI KLN':YTM, 
              'CHEUNG SHA WAN KLN':SSP, 
              'JORDAN KLN':YTM,
              'TSEUNG KWAN O NT':SK, 
              'CENTRAL HK':CNW, 
              'TSUEN WAN WEST NT':TW,
              'YUEN LONG NT':YL, 
              'KOWLOON CITY KLN':KC, 
              'TUEN MUN NT':TM,
              'TUEN MUN TAI LAM NT':TM, 
              'NGAU TAU KOK KLN':KTO, 
              'SAN TIN NT':YL,
              'SHEUNG WAN HK':CNW, 
              'KOWLOON TONG KLN':KC, 
              'TUNG CHUNG NT':ID,
              'HO MAN TIN KLN':KC, 
              'LOHAS PARK NT':SK, 
              'POK FU LAM HK':SD,
              'SHAU KEI WAN HK':ED, 
              'ABERDEEN HK':SD, 
              'TAI HANG HK':WC, 
              'SIU SAI WAN HK':ED,
              'KWUN TONG KLN':KTO, 
              'REPULSE BAY HK':SD, 
              'LAM TIN KLN':KTO, 
              'MA WAN NT':TW,
              'SAN PO KONG KLN':WTS, 
              'CHAI WAN HK':ED, 
              'LAI CHI KOK KLN':SSP,
              'DIAMOND HILL KLN':WTS, 
              'MUI WO NT':ID, 
              'DISCOVERY BAY NT':ID,
              'SHUN LEE KLN':WTS, 
              'KOWLOON BAY KLN':KTO, 
              'TAI KOK TSUI KLN':YTM,
              'AP LEI CHAU HK':SD, 
              'HUNG HOM BAY KLN':YTM, 
              'TAI PO MARKET NT':TP,
              'TAI WAI NT':ST, 
              'TIU KENG LENG NT':SK, 
              'THE PEAK HK':CNW, 
              'HAPPY VALLEY HK':WC,
              'CHOI HUNG KLN':WTS, 
              'MONGKOK KLN':YTM, 
              'NORTH POINT HK':ED, 
              'FO TAN NT':ST,
              'HUNG SHUI KIU NT':YL, 
              'SAI KUNG NT':SK, 
              'SHEK KIP MEI KLN':SSP,
              'SAI YING PUN HK':CNW, 
              'TAI PO KAU NT':TP, 
              'PRINCE EDWARD KLN':YTM,
              'KAM TIN NT':YL, 
              'SHEK TONG TSUI HK':CNW, 
              'TIN HAU HK':ED, 
              'SAI WAN HO HK':ED,
              'CHUNG HOM KOK HK':SD, 
              'SHOUSON HILL HK':SD, 
              'FORTRESS HILL HK':ED,
              'TING KAU NT':TW, 
              'UNIVERSITY NT':TP, 
              'LOK FU KLN':WTS, 
              'WONG CHUK HANG HK':SD,
              'CHOI WAN KLN':WTS, 
              'HA TSUEN NT':YL, 
              'LAI KING NT':KTI, 
              'PING SHAN NT':YL,
              'LAM TEI NT':TM, 
              'LANTAU ISLAND NT':ID, 
              'KWU TUNG':ND, 
              'TAI TAM HK':SD,
              'KWAI HING NT':KTI, 
              'SIU LAM NT':TM, 
              'TAI O NT':ID, 
              'SHEK O HK':SD,
              'LAU FAU SHAN NT':YL}

In [20]:
len(district_dict.keys())

111

In [21]:
def map_dist(arr):
    # size = len(arr)
    res = []
    for i in arr:
        res.append(district_dict[i])
    return res

In [22]:
# df1['district_name'] = map_dist(df1['district'].to_numpy())

In [23]:
df2['district_name'] = map_dist(df2['district'].to_numpy())

In [24]:
df2.head()

,membership_level,order_date,delivery_date,app_type,device_type,area,district,estate_name_en,estate_name_chi,housing_type,...,sub_cat_4_name_chi,store_name_chi,pickup_date,size,weight,height,width,manufacturer_country_chi,mean_order_value,district_name
2914,VIP,2022-04-19,2022-04-21,MAIN,Android,NT,TIN SHUI WAI NT,GRANDEUR TERRANCE,俊宏軒,Public,...,日拋,kcIsF6Av9WX/lzx9cLBNom1rpW4kvx7AUs317LsOHkE=,2022-04-20 03:00:00 UTC,INTL_-4.25,80g,200mm,60mm,愛爾蘭,424.5,Yuen Long District
2926,VIP,2022-04-19,2022-04-21,MAIN,iOS,NT,TAI WO NT,TAI WO ESTATE,太和邨,Public,...,月拋,kcIsF6Av9WX/lzx9cLBNom1rpW4kvx7AUs317LsOHkE=,2022-04-20 03:00:00 UTC,INTL_-1.5,17.2g,200mm,60mm,新加坡,724.5,Tai Po District
2928,VIP,2022-04-19,2022-04-24,MAIN,iOS,KLN,TAI KOK TSUI KLN,HOI FU COURT,海富苑,HOS,...,日拋,kcIsF6Av9WX/lzx9cLBNom1rpW4kvx7AUs317LsOHkE=,2022-04-23 09:00:00 UTC,INTL_-2.00,80g,50mm,30mm,韓國,674.5,Yau Tsim Mong District
2930,VIP,2022-04-19,2022-04-24,MAIN,Android,HK,ABERDEEN HK,SHEK PAI WAN ESTATE,石排灣,Public,...,日拋,kcIsF6Av9WX/lzx9cLBNom1rpW4kvx7AUs317LsOHkE=,2022-04-23 03:00:00 UTC,NaN,0.2g,5mm,5mm,新加坡,374.5,Southern District
2932,NORMAL,2022-04-19,2022-04-21,MAIN,iOS,KLN,HO MAN TIN KLN,OI MAN UPPER ESTATE,愛民邨上邨,Public,...,日拋,kcIsF6Av9WX/lzx9cLBNom1rpW4kvx7AUs317LsOHkE=,2022-04-20 03:00:00 UTC,NaN,30g,200mm,60mm,日本,674.5,Kowloon City District


In [25]:
# df1.to_csv('data_districtNotNA.csv')

In [26]:
df2.to_csv('data_district_estate_notNA.csv')

# 探索分析

## 各地区订单总价值排名

In [27]:
groups_district = df2.groupby(['district_name'],as_index=False)['mean_order_value'].sum()
groups_district.sort_values(by='mean_order_value', ascending=False)

,district_name,mean_order_value
1,Eastern District,245176.0
8,Sha Tin District,214002.0
15,Wong Tai Sin District,180849.0
5,Kwun Tong District,159453.0
17,Yuen Long District,152741.0
7,Sai Kung District,148897.5
4,Kwai Tsing District,106038.0
13,Tuen Mun District,103237.5
3,Kowloon City District,96527.0
12,Tsuen Wan District,96016.0


## 订单总价值最高的地区的各小区订单总价值排名

In [28]:
groups_district = df2.groupby(['district_name', 'estate_name_chi'],as_index=False)['mean_order_value'].sum()
groups_district[groups_district['district_name'] == 'Eastern District'].sort_values(by='mean_order_value', ascending=False)

,district_name,estate_name_chi,mean_order_value
7,Eastern District,太古城,54713.0
6,Eastern District,城市花園,52965.5
13,Eastern District,康怡花園,15440.0
18,Eastern District,杏花邨,13019.5
14,Eastern District,康翠台,10271.5
21,Eastern District,翠灣邨,9270.5
11,Eastern District,小西灣邨,8669.5
16,Eastern District,愛蝶灣,8496.5
25,Eastern District,興華邨,6871.5
20,Eastern District,環翠邨,6796.0


## 各地区订单平均价值排名

In [29]:
groups_district = df2.groupby(['district_name'],as_index=False)['mean_order_value'].mean()
groups_district.sort_values(by='mean_order_value', ascending=False)

,district_name,mean_order_value
1,Eastern District,1238.262626
15,Wong Tai Sin District,1189.796053
14,Wan Chai District,1141.166667
11,Tai Po District,1035.302469
3,Kowloon City District,1005.489583
17,Yuen Long District,909.172619
16,Yau Tsim Mong District,905.534483
8,Sha Tin District,869.926829
10,Southern District,868.829897
4,Kwai Tsing District,855.145161


## 订单总价值最高的地区的各小区订单平均价值排名

In [30]:
groups_district = df2.groupby(['district_name', 'estate_name_chi'],as_index=False)['mean_order_value'].mean()
groups_district[groups_district['district_name'] == 'Eastern District'].sort_values(by='mean_order_value', ascending=False)

,district_name,estate_name_chi,mean_order_value
6,Eastern District,城市花園,2787.657895
7,Eastern District,太古城,2279.708333
23,Eastern District,興東邨,2149.500000
3,Eastern District,南豐新邨,1474.500000
14,Eastern District,康翠台,1467.357143
9,Eastern District,富欣花園,1449.500000
16,Eastern District,愛蝶灣,1213.785714
18,Eastern District,杏花邨,1183.590909
24,Eastern District,興民邨,1053.666667
21,Eastern District,翠灣邨,1030.055556


## 各地区订单量排名

In [31]:
groups_district = df2.groupby(['district_name'],as_index=False)['mean_order_value'].count()
groups_district.sort_values(by='mean_order_value', ascending=False)

,district_name,mean_order_value
8,Sha Tin District,246
7,Sai Kung District,205
1,Eastern District,198
5,Kwun Tong District,194
17,Yuen Long District,168
15,Wong Tai Sin District,152
13,Tuen Mun District,125
4,Kwai Tsing District,124
12,Tsuen Wan District,118
9,Sham Shui Po District,109


## 订单量最高的地区的各小区订单量排名

In [32]:
groups_district = df2.groupby(['district_name', 'estate_name_chi'],as_index=False)['mean_order_value'].count()
groups_district[groups_district['district_name'] == 'Sha Tin District'].sort_values(by='mean_order_value', ascending=False)

,district_name,estate_name_chi,mean_order_value
142,Sha Tin District,名城,21
153,Sha Tin District,沙田第一城,21
147,Sha Tin District,愉翠苑,20
146,Sha Tin District,恒安邨,13
163,Sha Tin District,聽濤雅苑,12
171,Sha Tin District,隆亨邨,12
168,Sha Tin District,錦泰苑,11
152,Sha Tin District,水泉澳邨,10
175,Sha Tin District,駿洋邨,9
148,Sha Tin District,新港城,9


## 各地区订单等待时间排名

（使用tableau计算更容易）

In [33]:
# groups_district = df2.groupby(['district_name'],as_index=False)['waiting_time'].mean()
# groups_district.sort_values(by='waiting_time', ascending=False)

## 平均等待时间最高的地区的各小区订单平均等待时间排名

In [34]:
# groups_district = df2.groupby(['district_name', 'estate_name_chi'],as_index=False)['waiting_time'].mean()
# groups_district[groups_district['district_name'] == 'Wan Chai District'].sort_values(by='waiting_time', ascending=False)

In [35]:
# groups_district = df2.groupby(['district_name', 'estate_name_chi'],as_index=False)['waiting_time'].mean()
# groups_district[groups_district['district_name'] == 'North District'].sort_values(by='waiting_time', ascending=False)

In [36]:
# groups_district = df2.groupby(['district_name', 'estate_name_chi'],as_index=False)['waiting_time'].mean()
# groups_district[groups_district['district_name'] == 'Yau Tsim Mong District'].sort_values(by='waiting_time', ascending=False)

## 各区平均运费排名

In [43]:
df2[df2['delivery_cost'] != 0]#['delivery_cost']

,membership_level,order_date,delivery_date,app_type,device_type,area,district,estate_name_en,estate_name_chi,housing_type,...,sub_cat_4_name_chi,store_name_chi,pickup_date,size,weight,height,width,manufacturer_country_chi,mean_order_value,district_name
256448,VIP,2022-04-20,2022-04-22,MAIN,iOS,NT,MA ON SHAN NT,YAN ON ESTATE,欣安邨,Public,...,日拋,Ox8UIwj33PaiW52ET1y8ru+cZw8R4BRTfimIK34ygw0=,2022-04-21 03:00:00 UTC,NaN,70g,50mm,25mm,美國,374.5,Sha Tin District
257216,NORMAL,2022-04-20,2022-04-22,MAIN,iOS,NT,TIN SHUI WAI NT,TIN HANG ESTATE,天恆邨,Public,...,日拋,Ox8UIwj33PaiW52ET1y8ru+cZw8R4BRTfimIK34ygw0=,2022-04-21 03:00:00 UTC,NaN,70g,50mm,25mm,韓國,374.5,Yuen Long District
110267,NORMAL,2022-04-21,2022-04-26,MAIN,Android,KLN,TSZ WAN SHAN KLN,TSZ LOK ESTATE,慈樂邨,Public,...,月拋,kcIsF6Av9WX/lzx9cLBNom1rpW4kvx7AUs317LsOHkE=,2022-04-25 03:00:00 UTC,INTL_-7.5,17.2g,4.5mm,2.5mm,新加坡,274.5,Wong Tai Sin District
97028,NORMAL,2022-04-23,2022-04-26,MAIN,Web,NT,SAN TIN NT,FAIRVIEW PARK,錦綉花園,Private,...,日拋,kcIsF6Av9WX/lzx9cLBNom1rpW4kvx7AUs317LsOHkE=,2022-04-25 03:00:00 UTC,INTL_-7.5,80g,200mm,60mm,愛爾蘭,574.5,Yuen Long District
97035,NORMAL,2022-04-23,2022-04-26,MAIN,Web,NT,SAN TIN NT,FAIRVIEW PARK,錦綉花園,Private,...,日拋,kcIsF6Av9WX/lzx9cLBNom1rpW4kvx7AUs317LsOHkE=,2022-04-25 03:00:00 UTC,INTL_-6,80g,200mm,60mm,愛爾蘭,574.5,Yuen Long District
97032,VIP,2022-04-25,2022-04-28,MAIN,iOS,NT,TUNG CHUNG NT,TUNG CHUNG CRESCENT,東堤灣畔,Private,...,日拋,S4voQadWzkNiCRx4lhdnyNSTQRdY2cm3d9641MCm+4E=,2022-04-27 03:00:00 UTC,INTL_-3.75,200g,50mm,40mm,愛爾蘭,324.5,Island District


In [46]:
groups_district = df2[df2['delivery_cost'] != 0].groupby(['district_name'],as_index=False)['delivery_cost'].mean()
groups_district.sort_values(by='delivery_cost', ascending=False)

,district_name,delivery_cost
2,Wong Tai Sin District,80
3,Yuen Long District,80
0,Island District,40
1,Sha Tin District,40


In [47]:
# groups_district = df2.groupby(['district_name'],as_index=False)['delivery_cost'].mean()
# groups_district.sort_values(by='delivery_cost', ascending=False)

## 平均运费最高的地区的各小区订单平均运费排名

In [49]:
groups_district = df2[df2['delivery_cost'] != 0].groupby(['district_name', 'estate_name_chi'],as_index=False)['delivery_cost'].mean()
groups_district[groups_district['district_name'] == 'Wong Tai Sin District'].sort_values(by='delivery_cost', ascending=False)

,district_name,estate_name_chi,delivery_cost
2,Wong Tai Sin District,慈樂邨,80


In [55]:
groups_district = df2[df2['delivery_cost'] != 0].groupby(['district_name', 'estate_name_chi'],as_index=False)['mean_order_value'].count()
groups_district[groups_district['district_name'] == 'Wong Tai Sin District'].sort_values(by='mean_order_value', ascending=False)

,district_name,estate_name_chi,mean_order_value
2,Wong Tai Sin District,慈樂邨,1


In [56]:
groups_district = df2[df2['delivery_cost'] != 0].groupby(['district_name', 'estate_name_chi'],as_index=False)['mean_order_value'].sum()
groups_district[groups_district['district_name'] == 'Wong Tai Sin District'].sort_values(by='mean_order_value', ascending=False)

,district_name,estate_name_chi,mean_order_value
2,Wong Tai Sin District,慈樂邨,274.5


In [50]:
groups_district = df2[df2['delivery_cost'] != 0].groupby(['district_name', 'estate_name_chi'],as_index=False)['delivery_cost'].mean()
groups_district[groups_district['district_name'] == 'Yuen Long District'].sort_values(by='delivery_cost', ascending=False)

,district_name,estate_name_chi,delivery_cost
3,Yuen Long District,天恆邨,80
4,Yuen Long District,錦綉花園,80


In [57]:
groups_district = df2[df2['delivery_cost'] != 0].groupby(['district_name', 'estate_name_chi'],as_index=False)['mean_order_value'].count()
groups_district[groups_district['district_name'] == 'Yuen Long District'].sort_values(by='mean_order_value', ascending=False)

,district_name,estate_name_chi,mean_order_value
4,Yuen Long District,錦綉花園,2
3,Yuen Long District,天恆邨,1


In [58]:
groups_district = df2[df2['delivery_cost'] != 0].groupby(['district_name', 'estate_name_chi'],as_index=False)['mean_order_value'].sum()
groups_district[groups_district['district_name'] == 'Yuen Long District'].sort_values(by='mean_order_value', ascending=False)

,district_name,estate_name_chi,mean_order_value
4,Yuen Long District,錦綉花園,1149.0
3,Yuen Long District,天恆邨,374.5
